In [27]:
cars = [['med', 'low', '3', '4', 'med', 'med'], ['med', 'vhigh', '4', 'more', 'small', 'high'],
        ['high', 'med', '3', '2', 'med', 'low'], ['med', 'low', '4', '4', 'med', 'low'], 
        ['med', 'low', '5more', '2', 'big', 'med'], ['med', 'med', '2', 'more', 'big', 'high'], 
        ['med', 'med', '2', 'more', 'med', 'med'], ['vhigh', 'vhigh', '2', '2', 'med', 'low'],
        ['high', 'med', '4', '2', 'big', 'low'], ['low', 'low', '2', '4', 'big', 'med']]

car_labels = ['acc', 'acc', 'unacc', 'unacc', 'unacc', 'vgood', 'acc', 'unacc', 'unacc', 'good']

lista = []
for data in range(len(cars)):
    lista.append(cars[data][3])
x = list(set(lista))
x

['4', '2', 'more']

In [28]:
#dataset reprezinta lista cu features, labels reprezinta lista cum sunt catalogate masinele, column este spliting features in subsets
#acest block de cod este pentru a crea top tree feature dupa care se face split-ul 
#set() interesanta aceasta functie selecteaza toate elementele unice
def split(dataset, labels, column):
    data_subsets = []
    label_subsets = []
    counts = list(set([data[column] for data in dataset]))# selectarea coloanei din tabel(feature) dupa care se face split-ul
    for k in counts:
        new_data_subset = []
        new_label_subset = []
        for i in range(len(dataset)):
            if dataset[i][column] == k:
                new_data_subset.append(dataset[i])#prin aceasta sintaxa creeam subset in functie de atributele din feature
                new_label_subset.append(labels[i])#la fel si aici in functie de feature attributes creeam subset pentru label
        data_subsets.append(new_data_subset)#aici adaugam toate subseturile intr-o singura lista 
        label_subsets.append(new_label_subset)
    return data_subsets, label_subsets
#print(split(cars, car_labels, 3))

In [29]:
from collections import Counter
import random
import numpy as np

In [30]:
#calculate gini_impurity 1 - squared percentage of each label in the set 
def gini(dataset):
    impurity = 1
    label_count = Counter(dataset)
    for label in label_count:
        probability = label_count[label] / len(dataset)
        impurity -= probability**2
    return impurity
#print(gini(car_labels))

In [31]:
#aceasta metoda folosita pentru a identifica care feature este potrivit pentru split, aceasta metoda masoara diferenta dintre 
# gini impurity of the data before and after the split, se face pentru fiecare feauture
# impurity este 0 atunci cat toate elementele dintr-un set sunt din acelasi feature indiferent de numarul lor
# weighted information reflecta faptu ca marimea setului conteaza, example if set before split is 10 and subset 2 then 2/10 * impurity
#the importance of small wets is lowered 
def info_gain(starting_labels, split_labels):
    info_gains = gini(starting_labels) 
    for subset in split_labels:
        weight  = len(subset)/len(starting_labels) * gini(subset)
        info_gains -= weight
    return info_gains

In [32]:
split_data, split_labels = split(cars, car_labels, 3)
print(split_data, split_labels)
print(info_gain(car_labels, split_labels))

[[['med', 'low', '3', '4', 'med', 'med'], ['med', 'low', '4', '4', 'med', 'low'], ['low', 'low', '2', '4', 'big', 'med']], [['high', 'med', '3', '2', 'med', 'low'], ['med', 'low', '5more', '2', 'big', 'med'], ['vhigh', 'vhigh', '2', '2', 'med', 'low'], ['high', 'med', '4', '2', 'big', 'low']], [['med', 'vhigh', '4', 'more', 'small', 'high'], ['med', 'med', '2', 'more', 'big', 'high'], ['med', 'med', '2', 'more', 'med', 'med']]] [['acc', 'unacc', 'good'], ['unacc', 'unacc', 'unacc', 'unacc'], ['acc', 'vgood', 'acc']]
0.30666666666666675


In [33]:
#prin acest block de cod se cauta cel mai adecvat feature dupa care sa facem split-ul range(0,6) inseamna fiecare coloana din tabel
#blocul acesta are legatura stransa cu metoda split()
for feature in range(0, 6):
    split_data, split_labels = split(cars, car_labels, feature)
    print(info_gain(car_labels, split_labels))

0.2733333333333334
0.04000000000000001
0.10666666666666663
0.30666666666666675
0.15000000000000002
0.29000000000000004


In [34]:
#this function takes a set of datapoints and set of labels and returns the index of the feature that cause best split and info gain
def find_best_split(dataset, labels):
    best_feature = 0
    best_gain = 0
    for feature in range(len(dataset[0])):
        data_subsets, labels_subsets = split(dataset, labels, feature)
        gain = info_gain(labels, labels_subsets)
        if gain > best_gain:
            best_gain, best_feature = gain, feature
    return best_feature, best_gain
best_feature, best_gain = find_best_split(cars, car_labels)
print(best_feature, best_gain)

3 0.30666666666666675


In [35]:
#next function take as parameters dataset and labels 
def build_tree(data, labels):
      best_feature, best_gain = find_best_split(data, labels)
      if best_gain == 0:
        return Counter(labels)
      data_subsets, label_subsets = split(data, labels, best_feature)
      branches = []
      for i in range(len(data_subsets)):
            branch = build_tree(data_subsets[i], label_subsets[i])
            branches.append(branch)
      return branches
tree = build_tree(cars, car_labels)
print(tree)

[[Counter({'acc': 1}), Counter({'unacc': 1}), Counter({'good': 1})], Counter({'unacc': 4}), [Counter({'acc': 1}), Counter({'acc': 1}), Counter({'vgood': 1})]]


In [36]:
#function to Check whether variable or object is of dict type, list type, set type, tuple type, in this case
count = Counter(car_labels)
big = max(count.values())
print(count[big], big)
isinstance(count, dict)

0 5


True

In [37]:
indices = []
for i in range(0, 999):
    indices.append(random.randint(0, 999))
#print(indices)

In [38]:
indices = [random.randint(0, 999) for i in range(1000)]

In [39]:
6**0.5


2.449489742783178

In [40]:
lista = range(100)
print(type(random.choices(lista, k=4)))

<class 'list'>


In [57]:
lista1 = [2, 3, 4, 5]
weight = [0.1, 0.2, 0.5, 0.2]
print(len(lista1))
np.random.choice(lista1, size = 2, replace = False, p = weight )

4


array([4, 4])

In [65]:
max([], default = 0)
print(max(("python", "lua", "ruby"), key = len))

python


In [90]:
import operator #key-functions
weights = [("caruso", "aba"), ("principale", "petru")]
max(weights, key = operator.itemgetter(1))

('principale', 'petru')